# FYI directions, delete later

You are to build upon the predictive analysis (classification) that you already completed in the previous mini-project, adding additional modeling from new classification algorithms as well as more explanations that are inline with the CRISP-DM framework. You should use appropriate cross validation for all of your analysis (explain your chosen method of performance validation in detail). Try to use as much testing data as possible in a realistic manner (you should define what you think
is realistic and why). 

This report is worth 20% of the final grade. Please upload a report (one per team) with all code used, visualizations, and text in a single document. The format of the document can be PDF, *.ipynb, or HTML. You can write the report in whatever format you like, but it is easiest to turn in the rendered Jupyter notebook. The results should be reproducible using your report. Please carefully describe every assumption and every step in your report.

Dataset Selection

Select a dataset identically to the way you selected for the first project work week and mini-project.
You are not required to use the same dataset that you used in the past, but you are encouraged.
You must identify two tasks from the dataset to regress or classify. That is:

* two classification tasks OR
* two regression tasks OR
* one classification task and one regression task

For example, if your dataset was from the diabetes data you might try to predict two tasks: (1)
classifying if a patient will be readmitted within a 30 day period or not, and (2) regressing what the
total number of days a patient will spend in the hospital, given their history and specifics of the
encounter like tests administered and previous admittance.

# Lab 2: Zillow Dataset Classification 
MSDS 7331 Data Mining - Section 403 - Lab 2

Team: Ivelin Angelov, Yao Yao, Kaitlin Kirasich, Albert Asuncion

<a id="top"></a>
## Contents
* <a href="#Imports">Imports</a>
* <a href="#Define">Define and Prepare Class Variables</a>
* <a href="#Describe">Describe the Final Dataset</a>
* <a href="#Evaluation">Explain Evaluation Metrics</a>
* <a href="#Splits">Training and Testing Splits</a>
* <a href="#Models">Three Different Classification/Regression Models</a>
* <a href="#Analysis">Visualizations of Results and Analysis</a>
* <a href="#Advantages">Advantages of Each Model</a>
* <a href="#Attributes">Important Attributes</a>
* <a href="#Deployment">Deployment</a>
* <a href="#Exceptional">Exceptional Work</a>
* <a href="#References">References</a>
________________________________________________________________________________________________________
<a id="Imports"></a>
# Imports

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split


variables = pd.read_csv('../../datasets/variables.csv').set_index('name')
dataset = pd.read_csv('../../datasets/train.csv', low_memory=False)

variables = variables.loc[variables.index.isin(dataset.columns)]

'The dataset has %d rows and %d columns' % dataset.shape

'The dataset has 116761 rows and 54 columns'

<a href="#top">Back to Top</a>
<a id="Define"></a>
# Define and Prepare Class Variables
<b>10 points</b>

<i>
<b>Description:</b><br/>
Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.
</i>

## Regression task

In this dataset we will predict the value of taxamount

In [2]:
X = dataset.copy()
y = X['taxamount']
del X['taxamount']
# del X['taxvaluedollarcnt']????

dataset_reg = {
    'X': X,
    'y': y
}

## Classification task
I this task we are creating a variable: has_garage with posible values 1 or 0 representing has garage or doesn't have garage. The class is a little bit skewed with only 0.368 percent having garage.

In [3]:
X = dataset.copy()
y = (X['garagecarcnt'] > 0).astype(int)
del X['garagetotalsqft']
del X['garagecarcnt']

dataset_class = {
    'X': X,
    'y': y
}

y.sum() / len(X)

0.36825652401058573

<a href="#top">Back to Top</a>
<a id="Describe"></a>
# Describe the Final Dataset
<b>5 points</b>

<i>
<b>Description:</b><br/>
Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).
</i>

## List of all variables

In [4]:
def output_variables_table(variables):
    variables = variables.sort_index()
    rows = ['<tr><th>Variable</th><th>Type</th><th>Scale</th><th>Description</th></tr>']
    for vname, atts in variables.iterrows():
        atts = atts.to_dict()
        # add scale if TBD
        if atts['scale'] == 'TBD':
            if atts['type'] in ['nominal', 'ordinal']:
                uniques = dataset[vname].unique()
                uniques = list(uniques.astype(str))
                if len(uniques) < 10:
                    atts['scale'] = '[%s]' % ', '.join(uniques)
                else:
                    atts['scale'] = '[%s]' % (', '.join(uniques[:5]) + ', ... (%d More)' % len(uniques))
            if atts['type'] in ['ratio', 'interval']:
                atts['scale'] = '(%d, %d)' % (dataset[vname].min(), dataset[vname].max())
        row = (vname, atts['type'], atts['scale'], atts['description'])
        rows.append('<tr><td>%s</td><td>%s</td><td>%s</td><td>%s</td></tr>' % row)
    return HTML('<table>%s</table>' % ''.join(rows))

output_variables_table(variables)

Variable,Type,Scale,Description
airconditioningtypeid,nominal,"[0, 1, 5, 13, 11, 9, 3]",Type of cooling system present in the home (if any)
assessmentyear,interval,"(2015, 2015)",The year of the property tax assessment
bathroomcnt,ordinal,"[2.5, 1.0, 2.0, 1.5, 3.0, ... (23 More)]",Number of bathrooms in home including fractional bathrooms
bedroomcnt,ordinal,"[3, 2, 4, 5, 6, ... (17 More)]",Number of bedrooms in home
buildingqualitytypeid,ordinal,"[7, 4, 10, 1, 8, 12, 6, 11]",Overall assessment of condition of the building from best (lowest) to worst (highest)
calculatedbathnbr,ordinal,"[2.5, 1.0, 2.0, 1.5, 3.0, ... (23 More)]",Number of bathrooms in home including fractional bathroom
calculatedfinishedsquarefeet,ratio,"(0, 10925)",Calculated total finished living area of the home
censustractandblock,nominal,"[61110023618600.0, 61110015230000.0, 61110006841300.0, 61110052978700.0, 61110040395800.0, ... (445 More)]",Census tract and block ID combined - also contains blockgroup assignment by extension
city,nominal,"[nan, CORNELIUS, DAMASCUS, BORING, CLACKAMAS, ... (231 More)]",USPS offical city name(s)
finishedsquarefeet12,ratio,"(0, 6615)",Finished living area


<a href="#top">Back to Top</a>
<a id="Evaluation"></a>
# Explain Evaluation Metrics
<b>10 points</b>

<i>
<b>Description:</b><br/>
Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.
</i>

For our evaluation, we will be taking into account the accuracy and F-measure.  In order to compute the F-measure, we will need the precision and recall.  Because F-measure is a weighted average of these, we think a better F-measure score means the model has a better precision and recall.

Accuracy is the number of true positives and true negatives divided by the total results.  We want to look at accuracy because we it is a good general statistic for our results.

Precision is the number of true positives divided by the total number of positive results we found.  In other words, precision is how well we can predict a positive correctly.  We care about this because we think this value will reflect the quality of our models.

Recall is the number of true positives divided by the sum of true positives and false negatives.  The consequences of type 2 errors, predicting a false negative, are not extreme so we think recall is an appropriate measure of completeness.

Finally, we will also use F-measure which is essentially a weighted average of the precision and recall into one a simple statistic. This will be a number between 0 and 1 where closer to 1 is better and approaching 0 is worse.

<a href="#top">Back to Top</a>
<a id="Imports"></a>
# Training and Testing Splits
<b>10 points</b>

<i>
<b>Description:</b><br/>
Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.
</i>

# TODO build chart of train size while accuracy continues to increase

We have a large dataset so 10 fold cross validation may take too long.  Because of this, we will be splitting our dataset into 80% train and 20% test.

<a href="#top">Back to Top</a>
<a id="Models"></a>
# Three Different Classification/Regression Models
<b>20 points</b>

<i>
<b>Description:</b><br/>
Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!
</i>

## Yao - 3 regression models

In [5]:
# Code for SVM here

## Kaitlin - 3 classification models

In [6]:
# Fix location_type
print(X['location_type'].unique())
X['location_type']=['' if x is np.nan else x for x in X['location_type']]
print(X['location_type'].unique())
X['location_type']=X.location_type.map({'PRIMARY':1,'ACCEPTABLE':2,'NOT ACCEPTABLE':0, '' : -1})

[nan 'PRIMARY' 'ACCEPTABLE' 'NOT ACCEPTABLE']
['' 'PRIMARY' 'ACCEPTABLE' 'NOT ACCEPTABLE']


In [7]:
X['location_type'].unique()

array([-1,  1,  2,  0])

In [8]:
# fix zipcode_type
print(X['zipcode_type'].unique())
X['zipcode_type']=['' if x is np.nan else x for x in X['zipcode_type']]
print(X['zipcode_type'].unique())
X['zipcode_type']=X.zipcode_type.map({'STANDARD':1,'PO BOX':2,'MILITARY':3, 'UNIQUE': 4, '' : -1})

[nan 'STANDARD' 'PO BOX' 'MILITARY' 'UNIQUE']
['' 'STANDARD' 'PO BOX' 'MILITARY' 'UNIQUE']


In [9]:
X['zipcode_type'].unique()

array([-1,  1,  2,  3,  4])

In [10]:
print(dataset_class['X'])

        Unnamed: 0  parcelid  airconditioningtypeid  bathroomcnt  bedroomcnt  \
0              415  17073783                      0          2.5           3   
1              481  17088994                      0          1.0           2   
2              523  17100444                      0          2.0           3   
3              533  17102429                      0          1.5           2   
4              580  17109604                      0          2.5           4   
5              672  17125829                      0          2.5           4   
6              673  17125829                      0          2.5           4   
7              674  17125829                      0          2.5           4   
8              675  17125829                      0          2.5           4   
9              676  17125829                      0          2.5           4   
10             727  17132911                      0          2.0           3   
11             728  17132911            

In [12]:
# fix transactiondate
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
print(X['transactiondate'].unique())
X['transactiondate'].apply(enc.fit_transform)

['2016-01-27' '2016-03-30' '2016-05-27' '2016-06-07' '2016-08-08'
 '2016-08-26' '2016-07-08' '2016-05-11' '2016-06-15' '2016-10-07'
 '2016-05-20' '2016-03-17' '2016-05-18' '2016-04-13' '2016-07-06'
 '2016-12-29' '2016-01-11' '2016-09-07' '2016-05-23' '2016-07-15'
 '2016-05-19' '2016-06-13' '2016-08-24' '2016-10-14' '2016-07-01'
 '2016-10-03' '2016-03-01' '2016-03-31' '2016-06-27' '2016-04-15'
 '2016-02-25' '2016-06-14' '2016-09-28' '2016-04-12' '2016-08-12'
 '2016-09-19' '2016-11-16' '2016-04-07' '2016-11-23' '2016-01-08'
 '2016-06-29' '2016-10-28' '2016-05-09' '2016-11-14' '2016-06-02'
 '2016-01-15' '2016-04-06' '2016-01-13' '2016-07-07' '2016-06-16'
 '2016-01-20' '2016-01-05' '2016-01-28' '2016-02-01' '2016-04-19'
 '2016-06-20' '2016-07-29' '2016-10-06' '2016-05-17' '2016-05-13'
 '2016-07-28' '2016-06-17' '2016-06-30' '2016-09-06' '2016-08-30'
 '2016-03-21' '2016-04-26' '2016-04-29' '2016-07-22' '2016-06-28'
 '2016-04-08' '2016-09-30' '2016-01-14' '2016-08-01' '2016-06-22'
 '2016-09-

ValueError: could not convert string to float: '2016-01-27'

In [ ]:
# fix city
print(X['city'].unique())
X['city']=['' if x is np.nan else x for x in X['city']]
print(X['city'].unique())
#X['city']=X.city.map({'STANDARD':1,'PO BOX':2,'MILITARY':3, 'UNIQUE': 4, '' : -1})

In [ ]:
# Code for train test split
# Start with 80/20 split.  Keep same test and train for each classification model for now
X = dataset_class['X']
y = dataset_class['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)


In [ ]:
# loading library
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics as mt

# instantiate learning model (k = 3)
knn = KNeighborsClassifier(n_neighbors=3)

# fitting the model
knn.fit(X_train, y_train)

# predict the response
#pred = knn.predict(X_test)

# evaluate accuracy
#print (mt.accuracy_score(y_test, pred))


In [ ]:
'''
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn import metrics as mt


# X and y already set above
yhat = np.zeros(y.shape) # we will fill this with predictions

scl = StandardScaler()
X = scl.fit_transform(X)

# create cross validation iterator
# cv = StratifiedKFold(n_splits=10)

# get a handle to the classifier object, which defines the type
#clf = KNeighborsClassifier(n_neighbors=3)

# now iterate through and get predictions, saved to the correct row in yhat
# NOTE: you can parallelize this using the cross_val_predict method
#for train, test in cv.split(X,y):
#    clf.fit(X[train],y[train])
#    yhat[test] = clf.predict(X[test])

#total_accuracy = mt.accuracy_score(y, yhat)
#print ('KNN accuracy', total_accuracy)
'''

<a href="#top">Back to Top</a>
<a id="Analysis"></a>
# Visualizations of Results and Analysis
<b>10 points</b>

<i>
<b>Description:</b><br/>
Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.
</i>

<a href="#top">Back to Top</a>
<a id="Advantages"></a>
# Advantages of Each Model
<b>10 points</b>

<i>
<b>Description:</b><br/>
Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.
</i>

<a href="#top">Back to Top</a>
<a id="Attributes"></a>
# Important Attributes
<b>10 points</b>

<i>
<b>Description:</b><br/>
Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.
</i>

<a href="#top">Back to Top</a>
<a id="Deployment"></a>
# Deployment
<b>5 points</b>

<i>
<b>Description:</b><br/>
How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?
</i>

<a href="#top">Back to Top</a>
<a id="Exceptional"></a>
# Exceptional Work
<b>10 points</b>

<i>
<b>Description:</b><br/>
You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the
performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?
</i>

<a href="#top">Back to Top</a>
<a id="References"></a>
# References:

- Kernels from Kaggle competition: https://www.kaggle.com/c/zillow-prize-1/kernels
- Scikitlearn logistic regression: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- Scikitlearn linear SVC: http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
- Stackoverflow pandas questions: https://stackoverflow.com/questions/tagged/pandas